In [270]:
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as torch_optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import copy

device = "mps" if torch.backends.mps.is_available() else "cpu"

In [271]:
df = pd.read_csv('./SeoulBikeData.csv', encoding= 'unicode_escape').drop(["Date", "Seasons", "Holiday"], axis=1)

In [272]:
dataset_cols = []
for col in df.columns:
    col_name = col
    col_name.replace(" ", "_")
    col_name = re.sub('[^A-Za-z0-9_]+', '', col_name)
    dataset_cols.append(col_name)

df.columns = dataset_cols

In [273]:
df["FunctioningDay"] = (df["FunctioningDay"] == "Yes").astype(int)
df = df[df["Hour"] == 12]
df = df.drop(["Hour","Windspeedms", "Visibility10m", "FunctioningDay"], axis=1)

df

,RentedBikeCount,TemperatureC,Humidity,DewpointtemperatureC,SolarRadiationMJm2,Rainfallmm,Snowfallcm
12,449,1.7,23,-17.2,1.11,0.0,0.0
36,479,4.3,41,-7.8,1.09,0.0,0.0
60,333,5.8,85,3.4,0.43,0.0,0.0
84,393,-0.3,38,-12.9,1.11,0.0,0.0
108,321,-2.3,25,-19.7,0.00,0.0,0.0
...,...,...,...,...,...,...,...
8652,752,9.1,57,1.0,1.69,0.0,0.0
8676,672,9.3,61,2.1,0.85,0.0,0.0
8700,635,7.9,14,-18.0,1.81,0.0,0.0
8724,661,5.6,27,-11.9,0.34,0.0,0.0


In [284]:
train, val, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])
X_train = train.drop(["RentedBikeCount"], axis=1).values
y_train = train['RentedBikeCount'].values

X_test = test.drop(["RentedBikeCount"], axis=1).values
y_test = test['RentedBikeCount'].values

X_train

array([[ 25.  ,  46.  ,  12.5 ,   2.93,   0.  ,   0.  ],
       [ 19.  ,  91.  ,  17.4 ,   0.67,   0.5 ,   0.  ],
       [ 13.4 ,  48.  ,   2.5 ,   1.64,   0.  ,   0.  ],
       ...,
       [ -8.6 ,  40.  , -19.8 ,   1.11,   0.  ,   0.  ],
       [ -3.2 ,  32.  , -17.5 ,   1.01,   0.  ,   0.  ],
       [ 11.9 ,  74.  ,   7.4 ,   1.84,   0.  ,   0.  ]])

In [285]:
X = torch.tensor(X_train, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)

# create a model that predicts the probability of survival
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(6, 300)
    self.linear2 = nn.Linear(300, 300)
    self.linear3 = nn.Linear(300, 1)

  def forward(self, x):
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.linear3(x))
    return x

model = Model()

In [288]:
learning_rate=0.03
num_epochs=50000
loss_fn = nn.MSELoss()
optimizer = torch_optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# train the model
losses = []
for epoch in range(num_epochs):
    # make predictions
    y_pred = model(X)

    # calculate loss
    loss = loss_fn(y_pred, y)
    losses.append(loss.item())

    # backpropagation
    loss.backward()

    # update model parameters
    optimizer.step()

    # reset gradients
    optimizer.zero_grad()

#     if epoch % 100 == 0:
#         print(f"Epoch {epoch}, loss = {loss.item()}")

# get the average loss
avg_loss = sum(losses) / len(losses)
print(f"Average loss: {avg_loss}")

# make predictions on the test set
with torch.no_grad():
  y_pred = model(X_test)
  y_pred = y_pred.numpy()

# Get output
y_pred = np.round(y_pred).astype(int)